<a href="https://colab.research.google.com/github/stellaevat/ontology-mapping/blob/main/colabs/load_and_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pronto owlready2 \
&& wget -O doid.obo https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/EfUC_RdrfZdOsOrtmNATjuoBPDaIkSTUMyxJXyO2KKC6yw?download=1 \
&& wget -O ncit.obo https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/ETmaJIC0fAlItdsp8WQxS_wBzKN_6x08EZrtsOxVnbzvSg?download=1 \
&& wget -O doid.owl https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/EVjp-NoZlZtDq42LqxZH08ABz2h7IWqdw2gxetzvA-u3hQ?download=1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 54.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.7 MB/s eta 0:00:00
  Created wheel for owlready2: filename=owlready2-0.44-cp310-cp310-linux_x86_64.whl size=24075245 sha256=1c018c8c3ce0f67b9418acf7f7802562559080700ad65a2b1e398464e0ace353
  Stored in directory: /root/.cache/pip/wheels/88/e1/04/583b0743b2907f091204baaa0aef9740f5ba5f3d2f6a5aa00d
Successfully built owlready2
--2023-10-23 16:05:28--  https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/EfUC_RdrfZdOsOrtmNATjuoBPDaIkSTUMyxJXyO2KKC6yw?download=1
Resolving gla-my.sharepoint.com (gla-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to gla-my.sharepoint.com (gla-my.sharepoin

In [2]:
import pronto
import owlready2

# pronto (OBO versions)

In [3]:
ncit = pronto.Ontology("ncit.obo")
doid = pronto.Ontology("doid.obo")

In [4]:
doid_rels = doid.relationships()
ncit_rels = ncit.relationships()

line = "\n----------------------------\n"

print(f"{line} DOID OWL relationships ({len(doid_rels)}) {line}")
for rel in doid_rels:
  print(rel)

print(f"{line} NCIT relationships ({len(ncit_rels)}) {line}")
for rel in ncit_rels:
  print(rel)


----------------------------
 DOID OWL relationships (2) 
----------------------------

Relationship('IDO:0000664', name='has material basis in')
Relationship('RO:is_a', name='is_a')

----------------------------
 NCIT relationships (97) 
----------------------------

Relationship('NCIT:R100', name='Disease_Has_Associated_Anatomic_Site')
Relationship('NCIT:R101', name='Disease_Has_Primary_Anatomic_Site')
Relationship('NCIT:R102', name='Disease_Has_Metastatic_Anatomic_Site')
Relationship('NCIT:R103', name='Disease_Has_Normal_Tissue_Origin')
Relationship('NCIT:R104', name='Disease_Has_Normal_Cell_Origin')
Relationship('NCIT:R105', name='Disease_Has_Abnormal_Cell')
Relationship('NCIT:R106', name='Disease_Has_Molecular_Abnormality')
Relationship('NCIT:R107', name='Disease_Has_Cytogenetic_Abnormality')
Relationship('NCIT:R108', name='Disease_Has_Finding')
Relationship('NCIT:R110', name='Disease_Is_Grade')
Relationship('NCIT:R111', name='Disease_May_Have_Normal_Tissue_Origin')
Relationship(

In [5]:
pneumo = doid.get_term("DOID:0050159")
pneumo_rels = pneumo.relationships

print("Name: ", pneumo.name)
print("Synonyms: ", pneumo.synonyms)
print("Equivalent to: ", pneumo.equivalent_to)

print("Superclasses:")
for sup in pneumo.superclasses(distance=1, with_self=False):
  print(sup)

print("Relations: ", len(pneumo_rels))
for rel in doid_rels:
  try:
    print(pneumo_rels[rel], end=", ")
  except:
    pass

Name:  lymphoid interstitial pneumonia
Synonyms:  frozenset()
Equivalent to:  TermSet({})
Superclasses:
Term('DOID:2797', name='idiopathic interstitial pneumonia')
Relations:  0


# owlready2 (OWL versions)

In [6]:
doid_owl = owlready2.get_ontology("doid.owl").load()

In [7]:
print("Axioms:\n", len(list(doid_owl.general_class_axioms())))
print(f"Annotation properties ({len(list(doid_owl.annotation_properties()))}):\n", list(doid_owl.annotation_properties()))
print("Properties:\n", len(list(doid_owl.properties())))
print("Classes:\n", len(list(doid_owl.classes())))
print("Individuals:\n", len(list(doid_owl.individuals())))

node = list(doid_owl.classes())[100]
print(f"Node {node.id[0]} properties:")
for prop in node.get_properties(node):
  for value in prop[node]:
    print(f"{prop.python_name} = {value}")

Axioms:
 0
Annotation properties (52):
 [rdf-schema.comment, rdf-schema.label, owl.deprecated, 1.1.description, 1.1.title, oboInOwl.date, oboInOwl.default-namespace, oboInOwl.hasOBOFormatVersion, oboInOwl.saved-by, obo.IAO_0000115, obo.IAO_0000119, oboInOwl.hasDbXref, obo.IAO_0000700, obo.IAO_0100001, doid.DO_AGR_slim, doid.DO_CFDE_slim, doid.DO_FlyBase_slim, doid.DO_GXD_slim, doid.DO_IEDB_slim, doid.DO_MGI_slim, doid.DO_RAD_slim, doid.DO_cancer_slim, doid.DO_infectious_disease_slim, doid.DO_rare_slim, doid.GOLD, doid.NCIthesaurus, doid.TopNodes_DOcancerslim, doid.gram-negative_bacterial_infectious_disease, doid.gram-positive_bacterial_infectious_disease, doid.sexually_transmitted_infectious_disease, doid.tick-borne_infectious_disease, doid.zoonotic_infectious_disease, 1.1.date, 1.1.type, terms.license, oboInOwl.SubsetProperty, oboInOwl.auto-generated-by, oboInOwl.created_by, oboInOwl.creation_date, oboInOwl.hasAlternativeId, oboInOwl.hasBroadSynonym, oboInOwl.hasExactSynonym, oboInOwl

In [8]:
all_props = list(doid_owl.properties())
hasDbXref = None
for prop in all_props:
  if prop.python_name == "hasDbXref":
    hasDbXref = prop
    break
print("Found: ", hasDbXref)

Found:  oboInOwl.hasDbXref


In [9]:
equivalences = {}
if hasDbXref:
  for node in list(doid_owl.classes()):
    if hasDbXref in node.get_properties(node):
      refs = [ref for ref in hasDbXref[node] if "NCI:" in ref]
      if refs:
        equivalences[node.id[0]] = refs

In [10]:
print("Equivalence relations: ", equivalences)
print("Total: ", len(equivalences))

Equivalence relations:  {'DOID:0001816': ['NCI:C3088', 'NCI:C9275'], 'DOID:0014667': ['NCI:C3235'], 'DOID:0040093': ['NCI:C114354'], 'DOID:0050120': ['NCI:C34792'], 'DOID:0050127': ['NCI:C128411'], 'DOID:0050144': ['NCI:C84797'], 'DOID:0050156': ['NCI:C35716'], 'DOID:0050157': ['NCI:C62586'], 'DOID:0050158': ['NCI:C35288'], 'DOID:0050214': ['NCI:C3155'], 'DOID:0050269': ['NCI:C35083'], 'DOID:0050328': ['NCI:C26734', 'NCI:C98921'], 'DOID:0050424': ['NCI:C3339'], 'DOID:0050425': ['NCI:C84501'], 'DOID:0050426': ['NCI:C79484'], 'DOID:0050427': ['NCI:C3452'], 'DOID:0050428': ['NCI:C3147'], 'DOID:0050429': ['NCI:C82865'], 'DOID:0050430': ['NCI:C3226'], 'DOID:0050431': ['NCI:C84571'], 'DOID:0050432': ['NCI:C97159'], 'DOID:0050433': ['NCI:C84711'], 'DOID:0050434': ['NCI:C84559'], 'DOID:0050436': ['NCI:C84906'], 'DOID:0050437': ['NCI:C84735'], 'DOID:0050438': ['NCI:C122805'], 'DOID:0050439': ['NCI:C85217'], 'DOID:0050440': ['NCI:C165527', 'NCI:C84708'], 'DOID:0050441': ['NCI:C84908'], 'DOID:005

# pronto (OBO versions)

In [11]:
sub_doid_to_ncit = {}
for doid_id, ncit_list in equivalences.items():
  if doid_id in doid.terms():
    for val in ncit_list:
      ncit_id = f"NCIT:{val.split(':')[1]}"
      if ncit_id in ncit.terms():
        ncit_node = ncit.get_term(ncit_id)
        ncit_parents = [p.id for p in ncit_node.superclasses(distance=1, with_self=False)]
        if ncit_parents:
          sub_doid_to_ncit[doid_id] = ncit_parents

In [12]:
print("Subsumption relations (DOID to NCIT): ", sub_doid_to_ncit)
print("Total: ", len(sub_doid_to_ncit))

Subsumption relations (DOID to NCIT):  {'DOID:0001816': ['NCIT:C176558', 'NCIT:C3088'], 'DOID:0014667': ['NCIT:C53547'], 'DOID:0040093': ['NCIT:C27153'], 'DOID:0050120': ['NCIT:C35439', 'NCIT:C53543'], 'DOID:0050127': ['NCIT:C35024'], 'DOID:0050144': ['NCIT:C28193', 'NCIT:C53543'], 'DOID:0050156': ['NCIT:C26869'], 'DOID:0050157': ['NCIT:C113159'], 'DOID:0050158': ['NCIT:C35714'], 'DOID:0050214': ['NCIT:C189005', 'NCIT:C3311'], 'DOID:0050269': ['NCIT:C35720', 'NCIT:C84353'], 'DOID:0050328': ['NCIT:C36285'], 'DOID:0050424': ['NCIT:C165470'], 'DOID:0050425': ['NCIT:C28193'], 'DOID:0050426': ['NCIT:C27555', 'NCIT:C28193'], 'DOID:0050427': ['NCIT:C156032', 'NCIT:C7757', 'NCIT:C8957'], 'DOID:0050428': ['NCIT:C34748', 'NCIT:C8957'], 'DOID:0050429': ['NCIT:C34909'], 'DOID:0050430': ['NCIT:C123329'], 'DOID:0050431': ['NCIT:C53543'], 'DOID:0050432': ['NCIT:C88412'], 'DOID:0050433': ['NCIT:C28286', 'NCIT:C53543'], 'DOID:0050434': ['NCIT:C34786', 'NCIT:C53543'], 'DOID:0050436': ['NCIT:C28193', 'NC

In [13]:
sub_ncit_to_doid = {}
for doid_id, ncit_list in equivalences.items():
  if doid_id in doid.terms():
    doid_node = doid.get_term(doid_id)
    doid_parents = [p.id for p in doid_node.superclasses(distance=1, with_self=False)]
    if doid_parents:
      for val in ncit_list:
        ncit_id = f"NCIT:{val.split(':')[1]}"
        if ncit_id in ncit.terms():
          sub_ncit_to_doid[ncit_id] = doid_parents

In [14]:
print("Subsumption relations (NCIT to DOID): ", sub_ncit_to_doid)
print("Total: ", len(sub_ncit_to_doid))

Subsumption relations (NCIT to DOID):  {'NCIT:C3088': ['DOID:175'], 'NCIT:C9275': ['DOID:175'], 'NCIT:C3235': ['DOID:4'], 'NCIT:C114354': ['DOID:8857'], 'NCIT:C34792': ['DOID:75'], 'NCIT:C128411': ['DOID:1352'], 'NCIT:C84797': ['DOID:9562'], 'NCIT:C35716': ['DOID:3770'], 'NCIT:C62586': ['DOID:2797'], 'NCIT:C35288': ['DOID:2797'], 'NCIT:C3155': ['DOID:0060032', 'DOID:0060033', 'DOID:439'], 'NCIT:C35083': ['DOID:121', 'DOID:1947', 'DOID:2059', 'DOID:2253', 'DOID:732'], 'NCIT:C26734': ['DOID:0080015', 'DOID:1459'], 'NCIT:C98921': ['DOID:0080015', 'DOID:1459'], 'NCIT:C3339': ['DOID:0050739', 'DOID:5295'], 'NCIT:C84501': ['DOID:331'], 'NCIT:C79484': ['DOID:37'], 'NCIT:C3452': ['DOID:0050737', 'DOID:225'], 'NCIT:C3147': ['DOID:3390'], 'NCIT:C82865': ['DOID:0050736', 'DOID:9182'], 'NCIT:C3226': ['DOID:0050736', 'DOID:3125'], 'NCIT:C84571': ['DOID:0060036'], 'NCIT:C97159': ['DOID:0060041'], 'NCIT:C84711': ['DOID:649'], 'NCIT:C84559': ['DOID:0050736', 'DOID:2843'], 'NCIT:C84906': ['DOID:0050737

In [15]:
# Maybe manipulate somehow so that when multiple hasDbXrefs exist, if any of them are parent/child of the other, keep only the one that's closest lexically or something like that?

In [16]:
with open("doid_to_ncit.csv", "w") as f:
  for doid_id, ncit_list in sub_doid_to_ncit.items():
    for ncit_id in ncit_list:
      f.write(f"{doid_id},{ncit_id}\n")

with open("ncit_to_doid.csv", "w") as f:
  for ncit_id, doid_list in sub_ncit_to_doid.items():
    for doid_id in doid_list:
      f.write(f"{ncit_id},{doid_id}\n")

In [20]:
# from google.colab import files
# files.download("doid_to_ncit.csv")
# files.download("ncit_to_doid.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>